# Hyperparameter tuning

In [11]:
import pandas as pd
import numpy as np

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score

from pprint import pprint


In [2]:
pd.set_option('display.max_rows', None, 'display.max_columns', None)

df = pd.read_csv('../data/results/results_basemodels.csv')

In [3]:
df.sort_values(by=['test_matthews_corrcoef', 'test_recall'], ascending=False).set_index(keys=['data type', 'model'])

test_balanced_accuracy  test_precision  \
data type                 model                                            
no corr -- simple         rf                    0.869643        0.923711   
no corr -- smote          rf                    0.885714        0.943473   
no corr -- simple         knn                   0.869281        0.935386   
raw data -- smote         grad_b                0.875595        0.943532   
pca -- smote              rf                    0.864881        0.934909   
pca transformed -- simple knn                   0.864519        0.935231   
raw data -- simple        knn                   0.863509        0.933880   
raw data -- smote         knn                   0.898913        0.981304   
                          rf                    0.872852        0.949532   
pca transformed -- simple grad_b                0.811310        0.894389   
pca -- smote              grad_b                0.841900        0.920643   
raw data -- simple        rf                    0.827976        0.907875   
                          grad_b                0.844462        0.921296   
no corr -- smote          grad_b                0.860352        0.937205   
pca transformed -- smote  knn                   0.884808        0.978409   
pca -- smote              knn                   0.880280        0.974000   
raw data -- smote         ada_b                 0.865114        0.945850   
pca -- simple             grad_b                0.817262        0.908741   
pca transformed -- smote  svm                   0.859990        0.957140   
pca -- simple             rf                    0.802976        0.893960   
no corr -- smote          knn                   0.869384        0.974242   
pca -- simple             ada_b                 0.806962        0.897112   
no corr -- simple         ada_b                 0.813509        0.902865   
                          grad_b                0.802252        0.899760   
pca -- simple             knn                   0.811128        0.904539   
raw data -- simple        ada_b                 0.813147        0.908676   
no corr -- smote          ada_b                 0.820885        0.922760   
raw data -- smote         svm                   0.822308        0.935498   
pca transformed -- simple rf                    0.779581        0.887185   
no corr -- simple         svm                   0.714286        0.844127   
no corr -- smote          svm                   0.795160        0.921429   
raw data -- simple        svm                   0.720238        0.848970   
raw data -- smote         tree                  0.802070        0.914470   
pca -- simple             tree                  0.781418        0.901269   
raw data -- simple        tree                  0.759757        0.887653   
raw data -- smote         lr                    0.801889        0.922627   
pca transformed -- smote  rf                    0.781004        0.908836   
no corr -- smote          tree                  0.783204        0.907561   
pca -- smote              tree                  0.772541        0.904508   
no corr -- simple         tree                  0.776708        0.895845   
pca -- smote              ada_b                 0.771118        0.894561   
pca transformed -- simple ada_b                 0.744695        0.870892   
raw data -- simple        lr                    0.738147        0.865037   
pca -- smote              svm                   0.762008        0.900317   
pca transformed -- smote  grad_b                0.766356        0.902724   
                          ada_b                 0.756418        0.890429   
no corr -- simple         lr                    0.725647        0.858883   
no corr -- smote          lr                    0.761465        0.912627   
pca transformed -- simple tree                  0.731056        0.868748   
pca transformed -- smote  lr                    0.745523        0.897606   
pca -- simple             svm                   0.684524        0.835619   
pca --

From all the results of the basemodels, we can see that the `Random Forest` without correlated data performs the best. It has a recall of `1.0`, which is very important, as we do not want to have false negatives. The `Random Forest` model also has the highest quality (MCC).

Unfortunately due to stochastic nature, performance can very. It is possible to set the random_state variable. However, this is not adviced.

## Tuning
Lets try to tune the model even more.

In [7]:
X_train, y_train = pd.read_pickle('../data/X_train.pkl'), pd.read_pickle('../data/y_train.pkl')
X_test, y_test = pd.read_pickle('../data/X_test.pkl'), pd.read_pickle('../data/y_test.pkl')

In [8]:
# get the data without correlation
no_corr_cols = ['MDVP:Fhi(Hz)', 'MDVP:Flo(Hz)', 'RPDE', 'DFA', 'spread2','D2', 'PPE']
X_no_corr_train = X_train[no_corr_cols]
X_no_corr_test = X_test[no_corr_cols]

In [14]:
# Create the baseline model
rf = make_pipeline(StandardScaler(), RandomForestClassifier())

cv = StratifiedKFold(n_splits=5)

# somehow matthews_corrcoef is not valid and it seems jupyter notebook uses another version python version?
scoring = ['balanced_accuracy', 'precision', 'recall', 'roc_auc', 'accuracy']

scores = cross_validate(rf, X_no_corr_train, y_train, cv=cv, scoring=scoring)

In [17]:
for test, score in scores.items():
    print(f'{test}:\t\t\t{score.mean():.3f}')

fit_time:			0.111
score_time:			0.022
test_balanced_accuracy:			0.840
test_precision:			0.908
test_recall:			0.992
test_roc_auc:			0.954
test_accuracy:			0.917


In [ ]:
# Another test
rf.fit(X_no_corr_train, y_train)

In [29]:
y_pred = rf.predict(X_no_corr_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 8  2]
 [ 1 28]]
              precision    recall  f1-score   support

           0       0.89      0.80      0.84        10
           1       0.93      0.97      0.95        29

    accuracy                           0.92        39
   macro avg       0.91      0.88      0.90        39
weighted avg       0.92      0.92      0.92        39



In [32]:
pprint(rf.get_params())

{'memory': None,
 'randomforestclassifier': RandomForestClassifier(),
 'randomforestclassifier__bootstrap': True,
 'randomforestclassifier__ccp_alpha': 0.0,
 'randomforestclassifier__class_weight': None,
 'randomforestclassifier__criterion': 'gini',
 'randomforestclassifier__max_depth': None,
 'randomforestclassifier__max_features': 'auto',
 'randomforestclassifier__max_leaf_nodes': None,
 'randomforestclassifier__max_samples': None,
 'randomforestclassifier__min_impurity_decrease': 0.0,
 'randomforestclassifier__min_samples_leaf': 1,
 'randomforestclassifier__min_samples_split': 2,
 'randomforestclassifier__min_weight_fraction_leaf': 0.0,
 'randomforestclassifier__n_estimators': 100,
 'randomforestclassifier__n_jobs': None,
 'randomforestclassifier__oob_score': False,
 'randomforestclassifier__random_state': None,
 'randomforestclassifier__verbose': 0,
 'randomforestclassifier__warm_start': False,
 'standardscaler': StandardScaler(),
 'standardscaler__copy': True,
 'standardscaler__wi

## Creating a random grid for random search
It is import to start with random search, to narrow down posible parameters for grid search.

In [41]:
n_estimators = np.arange(0, 2000, 100)
max_features = ['sqrt', 'log2', None]
max_depth = [int(x) for x in np.linspace(10, 110, num=11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstrap
}

pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['sqrt', 'log2', None],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': array([   0,  100,  200,  300,  400,  500,  600,  700,  800,  900, 1000,
       1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900])}


In [42]:
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, scoring='recall', n_iter=100, cv=5, verbose=1, n_jobs=-1)

In [43]:
rf_random.fit(X_no_corr_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/commons/conda/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
25 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "/commons/conda/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/commons/conda/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 383, in fit
    self._validate_estimator()
  File "/commons/conda/lib/python3.8/site-packages/sklearn/ensemble/_base.py", line 138, in _validate_estimator
    raise ValueError(
ValueError: n_estimators must be greater than 

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': array([   0,  100,  200,  300,  400,  500,  600,  700,  800,  900, 1000,
       1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900])},
                   scoring='recall', verbose=1)

In [44]:
rf_random.best_params_

{'n_estimators': 1700,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'log2',
 'max_depth': 30,
 'bootstrap': False}

In [4]:
import pickle
loaded_model = pickle.load(open('../data/model_saves/best_model.sav', 'rb'))

/commons/conda/lib/python3.8/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.1.1 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/commons/conda/lib/python3.8/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.1.1 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [45]:
best_random = rf_random.best_estimator_
best_random_ypred = best_random.predict(X_no_corr_test)

print(confusion_matrix(y_test, best_random_ypred))
print(classification_report(y_test, best_random_ypred))

[[ 8  2]
 [ 0 29]]
              precision    recall  f1-score   support

           0       1.00      0.80      0.89        10
           1       0.94      1.00      0.97        29

    accuracy                           0.95        39
   macro avg       0.97      0.90      0.93        39
weighted avg       0.95      0.95      0.95        39



In [10]:
roc_auc_score(y_test, best_random_ypred)

0.9


In [12]:
recall_score(y_test, best_random_ypred)

1.0

In [49]:
accuracy_score(y_test, best_random_ypred)

0.9487179487179487

## Conclusion
The results are great and there is no need to further optimize it for now. healthy patients (0) are not always classified correctly. However, patients with Parkinson's disease are not classified as false negatives. Therefore it is a great score. The accuracy is around 0.95 which is great and the goal is accomplished (recall and accuracy > 0.9). Using an ensemble method clearly improves the perofrmance.

In [46]:
rf_random.best_score_

1.0